[Inspo](https://medium.com/@ka.kaewnoparat/text-summarization-and-keyword-extraction-from-customer-reviews-in-french-part-3-3-172195f2816b)

In [ ]:
import spacy.cli

spacy.cli.download("fr_core_news_md")

In [ ]:
from pathlib import Path
import glob

import spacy
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("fr_core_news_md")
model = SentenceTransformer("dangvantuan/sentence-camembert-large")

In [2]:
my_file = open("../../data/french_stopword.txt", "r")
data = my_file.read()
my_file.close()
french_stopwords_list = data.replace('\n', ' ').split(" ")
french_stopwords_list.append("quelqu")

In [3]:
def slice_only_noun_token(ner, token_list):
    noun_slice_list = []
    proper_noun_slice_list = []
    for word_idx in range(len(token_list)):
        doc = ner(token_list[word_idx])  # we loop through each token in the candidate list

        for token in doc:  #then we check if each token falls into NOUN / PROPN parts of speech
            if token.pos_ == 'NOUN':
                noun_slice_list.append(token.text)
            elif token.pos_ == 'PROPN':
                proper_noun_slice_list.append(token.text)

    return noun_slice_list, proper_noun_slice_list

In [4]:
def top_n_extractor(
        model,
        input_text,
        noun_list,
        top_n=3):
    text_encoded = model.encode(input_text)
    noun_candidate_list_encoded = model.encode(noun_list)
    distances = cosine_similarity(text_encoded, noun_candidate_list_encoded)
    keywords = [noun_list[index] for index in distances.argsort()[0][-top_n:]]

    return keywords

In [6]:
df = pd.read_csv('../../data/filtered_cds_data.csv', sep=',')

In [ ]:
index = 0
for url in df['URL']:
    input_text = [df['Anfang des Briefes'][index]]
    vectorizer = CountVectorizer(stop_words=french_stopwords_list).fit(input_text)
    candidates = vectorizer.get_feature_names_out()

    noun_list, proper_list = slice_only_noun_token(
        ner=nlp,
        token_list=candidates
    )

    text_encoded = model.encode(input_text)
    noun_candidate_list_encoded = model.encode(noun_list)
    if len(noun_candidate_list_encoded) == 0:
        index += 1
        continue
    distances = cosine_similarity(text_encoded, noun_candidate_list_encoded)

    df['keyword_list'][index] = top_n_extractor(model=model, input_text=input_text, noun_list=noun_list)
    index += 1

In [38]:
df.to_csv('../../data/filtered_cds_data.csv')